## Utilizando as informações de torques

Até o final da segunda Sprint, os modelos não utilizavam as features a respeito dos torques, cor e motor ao prever falhas. Aqui, esses dados serão limpos e manipulados para que possam ser usados para o treinamento dos modelos, a fim de realizar um teste para comparar se esses dados ajudam ou não na predição.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [3]:
#Uso de três tabelas de resultados mergeados.

df_resultados = pd.read_csv("../data/df_resultados_full.csv")

In [ ]:
df_resultados.head()

In [7]:

#mostrar dados que tem o knr = 2023-2056234
df_resultados[df_resultados["KNR"] == "2023-2056234"]

,KNR,NAME,ID,STATUS,UNIT,VALUE_ID,VALUE,DATA
0,2023-2056234,042000,1,10,Deg,5.0,15601 ...,2024-05-16 13:21:03.000
1,2023-2056234,042000,1,10,mm,3.0,1.69 ...,2024-05-16 13:21:03.000
2,2023-2056234,042000,1,10,mm,2.0,1.98 ...,2024-05-16 13:21:03.000
3,2023-2056234,042000,1,10,mm,1.0,3.67 ...,2024-05-16 13:21:03.000
4,2023-2056234,042000,1,10,Nm,4.0,2.93 ...,2024-05-16 13:21:03.000
...,...,...,...,...,...,...,...,...
841,2023-2056234,SECTION_ALS_00000002,718,10,%,21.0,-2.23 ...,2024-05-20 21:10:52.000
842,2023-2056234,SECTION_ALS_00000002,718,10,%,19.0,-2.33 ...,2024-05-16 17:02:05.000
843,2023-2056234,SECTION_ALS_00000002,718,10,%,19.0,-2.44 ...,2024-05-20 21:10:52.000
844,2023-2056234,SECTION_ALS_00000002,718,10,%,21.0,-2.5 ...,2024-05-16 17:02:05.000


In [8]:
#mostrar quantas categorias temos na coluna unit
df_resultados["UNIT"].value_counts()

UNIT
Grad          1333865
Nm            1333095
min             66924
%               40558
mm               7632
\u00B0           7430
Clicks           7402
kg               7256
                 7043
V                6078
Deg              2544
Name: count, dtype: int64

In [75]:
#retirar dados que tem UNIT  == '          '
df_resultados = df_resultados[df_resultados["UNIT"] != '          ']

#retirar os espaços vazios após os nomes das categorias
df_resultados["UNIT"] = df_resultados["UNIT"].str.strip()

In [76]:
#mostrar quantas categorias temos na coluna unit
df_resultados["UNIT"].unique()

array(['Deg', 'mm', 'Nm', 'Grad', 'Clicks', 'min', '\\u00B0', '%', 'V',
       'kg'], dtype=object)

In [22]:
df_resultados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2812784 entries, 0 to 2819826
Data columns (total 8 columns):
 #   Column    Dtype  
---  ------    -----  
 0   KNR       object 
 1   NAME      object 
 2   ID        int64  
 3   STATUS    int64  
 4   UNIT      object 
 5   VALUE_ID  float64
 6   VALUE     object 
 7   DATA      object 
dtypes: float64(1), int64(2), object(5)
memory usage: 193.1+ MB


In [77]:
# Remover espaços extras e substituir valores não numéricos por NaN
df_resultados["VALUE"] = df_resultados["VALUE"].str.strip()  # Remove espaços em branco
df_resultados["VALUE"] = df_resultados["VALUE"].replace('', pd.NA)  # Substitui strings vazias por NaN
df_resultados["VALUE"] = df_resultados["VALUE"].str.replace(',', '.', regex=False)  # Substitui vírgulas por pontos

# Convertendo para float, erros serão convertidos para NaN
df_resultados["VALUE"] = pd.to_numeric(df_resultados["VALUE"], errors='coerce')

In [78]:
#Descobrir quantos Nan temos na coluna VALUE
df_resultados["VALUE"].isna().sum()

1298

In [79]:
#Tirar linhas que tem Nan na coluna VALUE
df_resultados = df_resultados.dropna(subset=["VALUE"])

In [80]:
# Agrupar por KNR e UNIT, e calcule a média dos VALUES
df_grouped = df_resultados.groupby(['KNR', 'UNIT'])['VALUE'].mean().reset_index()

# Criar um DataFrame com um índice de KNR e as colunas das unidades
df_pivot = df_grouped.pivot(index='KNR', columns='UNIT', values='VALUE').reset_index()

# Preencher os valores NaN com 0
df_pivot = df_pivot.fillna(0)
df_pivot.head()

UNIT,KNR,%,Clicks,Deg,Grad,Nm,V,\u00B0,kg,min,mm
0,2023-2016173,0.0,0.0,0.0,59.666667,6.606667,12.357,0.0,0.0,0.0,0.0
1,2023-2026098,0.0,0.0,0.0,104.250000,8.290000,12.848,0.0,0.0,0.0,0.0
2,2023-2026162,0.0,0.0,0.0,74.500000,8.327500,12.871,0.0,0.0,0.0,0.0
3,2023-2026175,0.0,0.0,0.0,98.250000,8.327500,12.489,0.0,0.0,0.0,0.0
4,2023-2026215,0.0,0.0,0.0,67.750000,8.327500,12.737,0.0,0.0,0.0,0.0


In [81]:
#Deixando todas as letras em maiúsculo para padronização
columns = df_pivot[['Clicks', 'Deg', 'Grad', 'Nm', 'V', 'kg', 'min', 'mm']]
df_pivot.columns = df_pivot.columns.str.upper()

df_pivot.head()

UNIT,KNR,%,CLICKS,DEG,GRAD,NM,V,\U00B0,KG,MIN,MM
0,2023-2016173,0.0,0.0,0.0,59.666667,6.606667,12.357,0.0,0.0,0.0,0.0
1,2023-2026098,0.0,0.0,0.0,104.250000,8.290000,12.848,0.0,0.0,0.0,0.0
2,2023-2026162,0.0,0.0,0.0,74.500000,8.327500,12.871,0.0,0.0,0.0,0.0
3,2023-2026175,0.0,0.0,0.0,98.250000,8.327500,12.489,0.0,0.0,0.0,0.0
4,2023-2026215,0.0,0.0,0.0,67.750000,8.327500,12.737,0.0,0.0,0.0,0.0


In [82]:
# Listar todas as colunas
print("Todas as colunas:", df_pivot.columns.tolist())

Todas as colunas: ['KNR', '%', 'CLICKS', 'DEG', 'GRAD', 'NM', 'V', '\\U00B0', 'KG', 'MIN', 'MM']


In [83]:
# Colunas a serem excluídas da normalização
columns_to_exclude = ['KNR', 'UNIT']

# Colunas a serem normalizadas
columns_to_normalize = df_pivot.columns.difference(columns_to_exclude)

# Verifique se as colunas a serem normalizadas são numéricas
numeric_columns = df_pivot[columns_to_normalize].select_dtypes(include=[float, int]).columns

# Normalizar as colunas
scaler = MinMaxScaler()
df_pivot[numeric_columns] = scaler.fit_transform(df_pivot[numeric_columns])

df_pivot.head()

UNIT,KNR,%,CLICKS,DEG,GRAD,NM,V,\U00B0,KG,MIN,MM
0,2023-2016173,1.0,0.0,0.0,0.264989,0.057512,0.908503,0.493034,0.0,1.0,0.0
1,2023-2026098,1.0,0.0,0.0,0.462990,0.072166,0.944602,0.493034,0.0,1.0,0.0
2,2023-2026162,1.0,0.0,0.0,0.330866,0.072493,0.946293,0.493034,0.0,1.0,0.0
3,2023-2026175,1.0,0.0,0.0,0.436343,0.072493,0.918208,0.493034,0.0,1.0,0.0
4,2023-2026215,1.0,0.0,0.0,0.300888,0.072493,0.936441,0.493034,0.0,1.0,0.0


In [86]:
#juntando com o df_resultados
df_resultados_final = df_merged.merge(df_pivot, on='KNR', how='left')
df_resultados_final.head()

,KNR,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,DATA,%,CLICKS,DEG,GRAD,NM,V,\U00B0,KG,MIN,MM
0,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PARALAMA DIANTEIRO 01. AMASSADO,2024-05-20 21:31:51.992,0.506392,1.0,0.750915,0.31188,0.425432,0.936882,0.564695,0.0,0.506803,0.933842
1,2023-2056234,T-CROSS,2R2R,DHS,HCEQS506,4013334,ZP7,PORTA TRASEIRA 04. CAROÇO / PICO,2024-05-20 21:30:22.772,0.506392,1.0,0.750915,0.31188,0.425432,0.936882,0.564695,0.0,0.506803,0.933842
2,2023-2056234,T-CROSS,2R2R,DHS,PCEQS620,2058570,ZP6,MÓDULO DO ACC (13) PEÇA INCORRETA ¨,2024-05-20 20:04:55.534,0.506392,1.0,0.750915,0.31188,0.425432,0.936882,0.564695,0.0,0.506803,0.933842
3,2023-2056234,T-CROSS,2R2R,DHS,PCEQS999,2087715,ESPC,TAMPA TRASEIRA 05. SUJEIRA NA PINTURA,2024-04-11 12:15:39.616,0.506392,1.0,0.750915,0.31188,0.425432,0.936882,0.564695,0.0,0.506803,0.933842
4,2023-3896001,T-CROSS,0Q0Q,,HCEQS203,2043971,ZP5A,TAMPA DIANTEIRA 05. SUJEIRA NA PINTURA,2024-06-05 17:38:51.565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
#quantos nan temos no df_resultados_final
df_resultados_final.isna().sum()

KNR             0
MODELO          0
COR             0
MOTOR           0
ESTACAO         0
USUARIO         0
HALLE           0
FALHA           0
DATA            0
%          314277
CLICKS     314277
DEG        314277
GRAD       314277
NM         314277
V          314277
\U00B0     314277
KG         314277
MIN        314277
MM         314277
dtype: int64

In [89]:
df_resultados_final.shape

(328106, 19)

In [90]:
#juntando com o df_resultados
df_resultados_teste = df_pivot.merge(df_merged, on='KNR', how='left')
df_resultados_teste.head()

,KNR,%,CLICKS,DEG,GRAD,NM,V,\U00B0,KG,MIN,MM,MODELO,COR,MOTOR,ESTACAO,USUARIO,HALLE,FALHA,DATA
0,2023-2016173,1.0,0.0,0.0,0.264989,0.057512,0.908503,0.493034,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-2026098,1.0,0.0,0.0,0.462990,0.072166,0.944602,0.493034,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-2026162,1.0,0.0,0.0,0.330866,0.072493,0.946293,0.493034,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-2026175,1.0,0.0,0.0,0.436343,0.072493,0.918208,0.493034,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-2026215,1.0,0.0,0.0,0.300888,0.072493,0.936441,0.493034,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Excluindo as colunas que não vamos mais usar
df_resultados_final = df_resultados_final.drop(columns=['UNIT', 'VALUE'])